In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=eeeed237b6b44f18f34e550192d0ae5a1448030eea57d18639a2bf756b1c39fa
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [4]:
spark = SparkSession \
        .builder \
        .appName("USVehicleAccidentAnalysis") \
        .getOrCreate()

In [7]:
df_charges = spark.read.csv("Data/Charges_use.csv", header=True)
df_damages = spark.read.csv("Data/Damages_use.csv", header=True)
df_endorse = spark.read.csv("Data/Endorse_use.csv", header=True)
df_pp = spark.read.csv("Data/Primary_Person_use.csv", header=True)
df_Units = spark.read.csv("Data/Units_use.csv", header=True)
df_restrict = spark.read.csv("Data/Restrict_use.csv", header=True)

In [6]:
df_Units.printSchema()

root
 |-- CRASH_ID: string (nullable = true)
 |-- UNIT_NBR: string (nullable = true)
 |-- UNIT_DESC_ID: string (nullable = true)
 |-- VEH_PARKED_FL: string (nullable = true)
 |-- VEH_HNR_FL: string (nullable = true)
 |-- VEH_LIC_STATE_ID: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- VEH_MOD_YEAR: string (nullable = true)
 |-- VEH_COLOR_ID: string (nullable = true)
 |-- VEH_MAKE_ID: string (nullable = true)
 |-- VEH_MOD_ID: string (nullable = true)
 |-- VEH_BODY_STYL_ID: string (nullable = true)
 |-- EMER_RESPNDR_FL: string (nullable = true)
 |-- OWNR_ZIP: string (nullable = true)
 |-- FIN_RESP_PROOF_ID: string (nullable = true)
 |-- FIN_RESP_TYPE_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_1_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_1_ID: string (nullable = true)
 |-- FORCE_DIR_1_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_2_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_2_ID: string (nullable = true)
 |-- FORCE_DIR_2_ID: string (nullable = true)
 |-- V

In [8]:
#1
df_pp.select('DEATH_CNT').distinct().show()
df_pp.select('CRASH_ID','UNIT_NBR','PRSN_NBR','PRSN_TYPE_ID','PRSN_GNDR_ID','DEATH_CNT').filter(df_pp.PRSN_GNDR_ID =='MALE').filter(df_pp.DEATH_CNT>0).count()

+---------+
|DEATH_CNT|
+---------+
|        0|
|        1|
+---------+



182

In [9]:
#2
df_Units.first()
df_Units.select('VEH_BODY_STYL_ID').distinct().show()
df_Units.filter(df_Units.VEH_BODY_STYL_ID.contains('MOTORCYCLE')).count()
#df_Units.filter(col('VEH_BODY_STYL_ID').contains('MOTORCYCLE')).count()

+--------------------+
|    VEH_BODY_STYL_ID|
+--------------------+
|                 BUS|
|                  NA|
|                 VAN|
|              PICKUP|
|SPORT UTILITY VEH...|
|PASSENGER CAR, 4-...|
|          FIRE TRUCK|
|               TRUCK|
|             UNKNOWN|
|           AMBULANCE|
|    POLICE CAR/TRUCK|
|          MOTORCYCLE|
|   YELLOW SCHOOL BUS|
|   POLICE MOTORCYCLE|
|PASSENGER CAR, 2-...|
|       TRUCK TRACTOR|
|      FARM EQUIPMENT|
|NEV-NEIGHBORHOOD ...|
|OTHER  (EXPLAIN I...|
|        NOT REPORTED|
+--------------------+



784

In [10]:
w = Window.partitionBy("VEH_BODY_STYL_ID").orderBy(col("count").desc())
df = df_Units.join(df_pp, on=['CRASH_ID'], how='inner'). \
            filter(~ df_Units.VEH_BODY_STYL_ID.isin(["NA", "UNKNOWN", "NOT REPORTED","OTHER  (EXPLAIN IN NARRATIVE)"])). \
            filter(~df_pp.PRSN_ETHNICITY_ID.isin(["NA", "UNKNOWN"])). \
            groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").count(). \
            withColumn("row", row_number().over(w)).filter(col("row") == 1).drop("row", "count")
df.count()

16

In [12]:
top_25_state_list = [row[0] for row in df_Units.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).
            groupby("VEH_LIC_STATE_ID").count().orderBy(col("count").desc()).limit(25).collect()]
                
top_10_used_vehicle_colors = [row[0] for row in df_Units.filter(df_Units.VEH_COLOR_ID != "NA").
            groupby("VEH_COLOR_ID").count().orderBy(col("count").desc()).limit(10).collect()]



df = df_charges.join(df_pp, on=['CRASH_ID'], how='inner'). \
            join(df_Units, on=['CRASH_ID'], how='inner'). \
            filter(df_charges.CHARGE.contains("SPEED")). \
            filter(df_pp.DRVR_LIC_TYPE_ID.isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])). \
            filter(df_Units.VEH_COLOR_ID.isin(top_10_used_vehicle_colors)). \
            filter(df_Units.VEH_LIC_STATE_ID.isin(top_25_state_list)). \
            groupby("VEH_MAKE_ID").count(). \
            orderBy(col("count").desc()).limit(5)
df.show()

+-----------+-----+
|VEH_MAKE_ID|count|
+-----------+-----+
|       FORD|19205|
|  CHEVROLET|16860|
|     TOYOTA|11822|
|      DODGE| 7935|
|     NISSAN| 7332|
+-----------+-----+

